## Wav2vec2 Fine-tuning notebook (Yasiru)

In [37]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install jiwer

## Prepare Data, Tokenizer, Feature Extractor

loading the dataset and taking a look at its structure.

In [2]:
%%capture
!apt install git-lfs

Loading the custom dataset

In [4]:
from datasets import load_dataset, DatasetDict

# Load each CSV file separately
train_data = load_dataset("csv", data_files="data/train.csv")
test_data = load_dataset("csv", data_files="data/test.csv")


# Create splits for each dataset
timit = DatasetDict({
    "train": train_data["train"],
    "test": test_data["train"],

})

# Access the splits using the keys
train_examples = timit["train"]
test_examples = timit["test"]


Found cached dataset csv (C:/Users/Yasiru/.cache/huggingface/datasets/csv/default-03a0222fdd129866/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset csv (C:/Users/Yasiru/.cache/huggingface/datasets/csv/default-b4019ba1a5962b11/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [38]:
%%capture
!apt install git-lfs

In [5]:

timit


DatasetDict({
    train: Dataset({
        features: ['file', 'text'],
        num_rows: 751
    })
    test: Dataset({
        features: ['file', 'text'],
        num_rows: 99
    })
})

we will only consider the transcribed text for fine-tuning.



display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [6]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(timit["train"].remove_columns(["file"]), num_examples=5)

,text
0,pop
1,boat
2,yyy
3,plate
4,pop


transcriptions look very clean

In [11]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [12]:
timit = timit.map(remove_special_characters)

Map:   0%|          | 0/751 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [13]:
show_random_elements(timit["train"].remove_columns(["file"]))

,text
0,okay
1,cake
2,pop
3,shape
4,yyy
5,plate
6,pop
7,cut
8,bead
9,yyy






write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars.
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once.

In [14]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])


Map:   0%|          | 0/751 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/4267 [00:00<?, ? examples/s]

union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.

In [15]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [16]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'c': 0,
 't': 1,
 ' ': 2,
 'y': 3,
 'i': 4,
 'm': 5,
 's': 6,
 'a': 7,
 'l': 8,
 'e': 9,
 'u': 10,
 'b': 11,
 'p': 12,
 'h': 13,
 'w': 14,
 'k': 15,
 'd': 16,
 'n': 17,
 'o': 18,
 'r': 19}

we also extracted the special characters `" "` and `'`.

In [17]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [18]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

22

Let's now save the vocabulary as a json file.

In [19]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [20]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [21]:
tokenizer.save_pretrained("./wav2vec2-base-mine/")

('./wav2vec2-base-mine/tokenizer_config.json',
 './wav2vec2-base-mine/special_tokens_map.json',
 './wav2vec2-base-mine/vocab.json',
 './wav2vec2-base-mine/added_tokens.json')

Next, we will create the feature extractor.

### Create Wav2Vec2 Feature Extractor

In [31]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)


feature_extractor.save_pretrained("./wav2vec2-base-mine/")

['./wav2vec2-base-mine/preprocessor_config.json']

In [34]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

processor.save_pretrained("./wav2vec2-base-mine/")




We will give the fine-tuned model the name `"wav2vec2-base-timit-demo"`.

In [24]:
timit["train"][0]

{'file': 'C:/Users/Yasiru/Downloads/dataset/data/dev/040221_278/040221_278.wav',
 'text': 'cut  '}

Alright, the audio file is saved in the `.WAV` format. There are a couple of python-based libraries to read and process audio files

`librosa` seems to be the most active and prominent library, but since it depends on `soundfile` for loading of audio files, we will just use `soundfile` directly in this notebook.


In [112]:
!pip install soundfile

In [26]:
%%capture
!apt install git-lfs

In [25]:

def speech_file_to_array_fn(batch):
    import soundfile as sf

    speech_array, sampling_rate = sf.read(batch["file"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

timit = timit.map(speech_file_to_array_fn, remove_columns=timit.column_names["train"], num_proc=4)

Map (num_proc=4):   0%|          | 0/751 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/99 [00:00<?, ? examples/s]

In [27]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

ipd.Audio(data=np.asarray(timit["train"][rand_int]["speech"]), autoplay=True, rate=16000)

It can be heard, that the speakers change along with their speaking rate, accent, etc.

In [28]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["target_text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["speech"]).shape)
print("Sampling rate:", timit["train"][rand_int]["sampling_rate"])

Target text: pop  
Input array shape: (15904,)
Sampling rate: 16000


Finally, we can process the dataset to the format expected by the model for training. We will again make use of the `map(...)` function.


In [32]:

def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
            len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

timit_prepared = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], batch_size=8, num_proc=4, batched=True)

Map (num_proc=4):   0%|          | 0/751 [00:00<?, ? examples/s]

NameError: name 'processor' is not defined

In [1]:
max_input_length_in_sec = 4.0

# add an input_length column to the dataset
timit = timit.map(lambda x: {"input_length": len(x["audio"])}, num_proc=4)

NameError: name 'timit' is not defined

## Training & Evaluation


- Define a data collator. In contrast to most NLP models, Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

In [98]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [77]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the 
predominant metric in ASR is the word error rate (WER)

In [78]:
from datasets import load_metric

wer_metric = load_metric("wer")

In [79]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Now, we can load the pretrained `Wav2Vec2` checkpoint.

In [39]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_hid.bias', 'project_hid.weight', 'project_q.weight', 'quantizer.weight_proj.weight', 'project_q.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti


Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [81]:
model.freeze_feature_extractor()

C:\Users\Yasiru\anaconda3\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:1639: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Timit dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

**Note**: If one wants to save the trained models in his/her google drive the commented-out `output_dir` can be used instead.

In [43]:

!pip install torchvision


Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are missing from the target environment:
  - cpuonly





Solving environment: ...working... failed


In [44]:
!pip install torch


In [82]:
import torch

if torch.cuda.is_available():
    print(torch.version.cuda)  # print CUDA version
    print(torch.backends.cudnn.version())  # print cuDNN version
else:
    print("No GPU available")

11.6
8302


In [58]:
!pip install --upgrade transformers

In [37]:
import torch
torch.cuda.empty_cache()

In [90]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./wav2vec2-base-mine/",
    group_by_length=True,
    per_device_train_batch_size=8,
    evaluation_strategy="steps",
    num_train_epochs=2,
    fp16=True,
    save_steps=250,
    eval_steps=500,
    logging_steps=100,
    learning_rate=1e-4,
    weight_decay=0.005,
    warmup_steps=100,
    save_total_limit=2,

)



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Now, all instances can be passed to Trainer and we are ready to start training!

In [91]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit_prepared["train"],
    eval_dataset=timit_prepared["test"],
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend




---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels


### Training

In [92]:
trainer.train()


C:\Users\Yasiru\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 751
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 188
  Number of trainable parameters = 90195872
C:\Users\Yasiru\anaconda3\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss


ValueError: `mask_length` has to be smaller than `sequence_length`, but got `mask_length`: 10 and `sequence_length`: 4`

In [37]:
!ls /kaggle/working/wav2vec2-base-mine

checkpoint-250		  special_tokens_map.json  vocab.json
preprocessor_config.json  tokenizer_config.json


In [38]:
!ls /kaggle/working/wav2vec2-base-mine/checkpoint-250

config.json   preprocessor_config.json	scheduler.pt	    training_args.bin
optimizer.pt  pytorch_model.bin		trainer_state.json


The final WER should be below 0.3 which is reasonable given that state-of-the-art phoneme error rates (PER) are just below 0.1 (see [leaderboard](https://paperswithcode.com/sota/speech-recognition-on-timit)) and that WER is usually worse than PER.

In [39]:
processor = Wav2Vec2Processor.from_pretrained("./wav2vec2-base-mine/")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [40]:
model = Wav2Vec2ForCTC.from_pretrained("./wav2vec2-base-mine/checkpoint-250")

Now, we will make use of the `map(...)` function to predict the transcription of every test sample and to save the prediction in the dataset itself. We will call the resulting dictionary `"results"`. 

**Note**: we evaluate the test data set with `batch_size=1` on purpose due to this [issue](https://github.com/pytorch/fairseq/issues/3227). Since padded inputs don't yield the exact same output as non-padded inputs, a better WER can be achieved by not padding the input at all.

In [41]:
def map_to_result(batch):
  model.to("cuda")
  input_values = processor(
      batch["speech"], 
      sampling_rate=batch["sampling_rate"], 
      return_tensors="pt"
  ).input_values.to("cuda")

  with torch.no_grad():
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  
  return batch

In [42]:
results = timit["test"].map(map_to_result)

Let's compute the overall WER now.

In [43]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))

Test WER: 1.000


In [44]:
show_random_elements(results.remove_columns(["speech", "sampling_rate"]))

,pred_str,target_text
0,ouluusual uds sful tlal ful w,the bungalow was pleasantly situated near the shore
1,ouluusual uds sful tlal ful w,the bungalow was pleasantly situated near the shore
2,ouluusual uds sful tlal ful w,the bungalow was pleasantly situated near the shore
3,ouluusual uds sful tlal ful w,the bungalow was pleasantly situated near the shore
4,ouluusual uds sful tlal ful w,the bungalow was pleasantly situated near the shore
5,ouluusual uds sful tlal ful w,the bungalow was pleasantly situated near the shore
6,ouluusual uds sful tlal ful w,the bungalow was pleasantly situated near the shore
7,ouluusual uds sful tlal ful w,the bungalow was pleasantly situated near the shore
8,ouluusual uds sful tlal ful w,the bungalow was pleasantly situated near the shore
9,ouluusual uds sful tlal ful w,the bungalow was pleasantly situated near the shore


In [45]:
model.to("cuda")
input_values = processor(timit["test"][0]["speech"], sampling_rate=timit["test"][0]["sampling_rate"], return_tensors="pt").input_values.to("cuda")

with torch.no_grad():
  logits = model(input_values).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] o o o o [PAD] [PAD] u l l [PAD] [PAD] [PAD] [PAD] [PAD] u u u u [PAD] [PAD] [PAD] u s s u u u u u u a l l l l l [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | [PAD] u u [PAD] [PAD] d [PAD] [PAD] [PAD] [PAD] s [PAD] [PAD] [PAD] | | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | | [PAD] s [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] f f u u l | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] t t l a a a l | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | f u u u u u l l l | [PAD] [PAD] | | | | [PAD] w w w'

In [46]:
input_values = processor(timit["test"][1]["speech"], sampling_rate=timit["test"][1]["sampling_rate"], return_tensors="pt").input_values.to("cuda")

with torch.no_grad():
  logits = model(input_values).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] o o o o [PAD] [PAD] u l l [PAD] [PAD] [PAD] [PAD] [PAD] u u u u [PAD] [PAD] [PAD] u s s u u u u u u a l l l l l [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | [PAD] u u [PAD] [PAD] d [PAD] [PAD] [PAD] [PAD] s [PAD] [PAD] [PAD] | | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | | [PAD] s [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] f f u u l | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] t t l a a a l | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | f u u u u u l l l | [PAD] [PAD] | | | | [PAD] w w w'

In [47]:
input_values = processor(timit["test"][2]["speech"], sampling_rate=timit["test"][2]["sampling_rate"], return_tensors="pt").input_values.to("cuda")

with torch.no_grad():
  logits = model(input_values).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] o o o o [PAD] [PAD] u l l [PAD] [PAD] [PAD] [PAD] [PAD] u u u u [PAD] [PAD] [PAD] u s s u u u u u u a l l l l l [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | [PAD] u u [PAD] [PAD] d [PAD] [PAD] [PAD] [PAD] s [PAD] [PAD] [PAD] | | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | | [PAD] s [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] f f u u l | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] t t l a a a l | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | f u u u u u l l l | [PAD] [PAD] | | | | [PAD] w w w'